# Kyber algorithm

## Description

This algorithm uses to provide security KEM in TLS, encryption, IoT etc, OpenSSL.

## Math Problem

### Lattice-problem

The lattice problem is a class of mathematical problems related to the geometric structure of lattices, which are grids of regularly spaced points in 
n
n-dimensional space. Key problems include the Shortest Vector Problem (SVP) and the Closest Vector Problem (CVP). SVP seeks the shortest non-zero vector in a lattice, while CVP finds the lattice point nearest to a given target. These problems are foundational in cryptography, especially in constructing post-quantum cryptographic schemes due to their computational hardness. Lattice problems are challenging even for quantum computers, making them a cornerstone for secure communication in the face of advancing quantum technology.

## Functions

n = ? - param of kyber can be different<br>
q = ? - param of kyber can be different<br>
m = x^n + x^(n-1) ... x^0 - message

A = random matrix<br>
s = 'small' polynom/matrix<br>
e = 'small' polynom/matrix<br>
t = part of the secret key<br>
(A, t) = Matrix A extended by t<br>
(e1, e2) = Matrix e1 extended by e2<br>
(0, t) = Empty Matrix 0 extended by t<br>
(u, v) = cipher text<br>


### Keygen
Key gen example.<br>
Create some A, s and e. s and e are 'small' numbers:<br>

```
t=A*s+e
```

(A, t) is the public key<br>
s is the secret key

### Encrypt

Encryption has a few steps:

```
r * (A, t) = res + (e1, e2) + (0, e3) = (u, v)
```

### Decrypt

Decryption example with math termin:

```
U*s ≈ v - m
``` 

With trick we can get m by subtracting v.<br>
U*s is close to v - m.

full kyber uses complex structure then numbers

In [3]:
import numpy as np

# imports

In [11]:
qCN = 7
nCN = 4

mCN = 5  # message 5

In [ ]:
# Test data:

A_test = [[8,-2,6,6], [8,3,4,10], [5,-4,6,-3], [9,5,-3,11]]
s_test = [2,1,0,-1]
e_test = [1,0,1,0]

r_test = [1,2,1,0]
e1e2_test = [0, 1, 0, 1, 1]



In [ ]:
def keygen(a_temp = [], s_temp = [], e_temp = []) -> tuple:
    A = np.random.randint(0, 10, size=(nCN, nCN))
    s = np.random.randint(0, 10, size=nCN)
    e = np.random.randint(-2, 2, size=nCN)

    if len(a_temp) > 0:
        A = a_temp
    if len(s_temp) > 0:
        s = s_temp
    if len(e_temp) > 0:
        e = e_temp

    print(np.dot(A, s))    
    t = np.dot(A, s) + e
    return (s, (A,t), e )

def encode(m, A, t):
    r = np.random.randint(0, 10, size=nCN)
    e1 = np.random.randint(-2, 2, size=nCN)
    e2 = [1]

    # At - A extended by t, e1e2 - e1 extended by e2, m0 - 0 vector extended by m
    At = np.concatenate((A, t.reshape(-1,1)), axis=1)
    res = np.dot(r, At)
    e1e2 = np.concatenate((e1, e2))
    m0 = np.concatenate((np.zeros(nCN), [m]))
    
    uv = res + e1e2 + m0 * qCN/2
    return uv

In [103]:
priv_key, pub_key, error = keygen(A_test, s_test, e_test)
# uv = encode(mCN, pub_key[0], pub_key[1])

print(f"n = {nCN}, q = {qCN}, m = {mCN}")
print(f"PrivateKey: {priv_key}")
print(f"PublicKey: {pub_key}")
print(f"Error: {error}")
# print(f"uv: {uv}")

n = 4, q = 7, m = 5
PrivateKey: [[2, 1, 0, -1]]
PublicKey: ([[8, -2, 6, 6], [8, 3, 4, 10], [5, -4, 6, -3], [9, 5, -3, 11]], array([[16, -6, 20, 11]]))
Error: [1, 0, 1, 0]
